# Calculate each ARG class and their carriers counts

In [12]:
import os
import pandas as pd
from collections import defaultdict

# 文件夹路径
folder_path = '/lomi_home/gaoyang/software/CompRanking/tmp_DSR/DSR/CompRanking_result'

# 初始化最终结果的字典
final_dict = defaultdict(lambda: defaultdict(int))

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.contigs_AMR_MOB_prediction.tsv'):
        filepath = os.path.join(folder_path, filename)
        
        # 提取样本名
        sample_name = filename.split('_')[1].split(".")[0]
        
        # 初始化当前文件的结果字典
        current_dict = defaultdict(lambda: [0, 0, 0, 0, 0])
        
        # 使用 chunksize 分块读取文件
        for chunk in pd.read_csv(filepath, sep='\t', chunksize=10000):
            # 按条件过滤数据
            filtered_chunk = chunk[chunk['CompRanking_MGE_prediction'].isin(['plasmid', 'phage', 'unclassified', 'IS', 'IE'])]
            
            # 预处理 ARG_class 列
            def preprocess_arg_class(arg_class):
                if arg_class == '-':
                    return None
                arg_class = arg_class.split('/')[0]
                arg_class = arg_class.split(':')[0]
                return arg_class
            
            filtered_chunk['ARG_class'] = filtered_chunk['ARG_class'].apply(preprocess_arg_class)
            filtered_chunk = filtered_chunk.dropna(subset=['ARG_class'])
            
            # 使用 groupby 和 size 进行计数
            grouped = filtered_chunk.groupby(['ARG_class', 'CompRanking_MGE_prediction']).size().reset_index(name='count')
            
            for _, row in grouped.iterrows():
                arg_class = row['ARG_class']
                mge_prediction = row['CompRanking_MGE_prediction']
                count = row['count']
                
                if mge_prediction == 'plasmid':
                    current_dict[arg_class][0] += count
                elif mge_prediction == 'phage':
                    current_dict[arg_class][1] += count
                elif mge_prediction == 'unclassified':
                    current_dict[arg_class][2] += count
                elif mge_prediction == 'IS':
                    current_dict[arg_class][3] += count
                elif mge_prediction == 'IE':
                    current_dict[arg_class][4] += count
        
        # 更新最终结果字典
        for arg_class, counts in current_dict.items():
            final_dict[arg_class][f'{sample_name}_x'] += counts[0] #plasmid
            final_dict[arg_class][f'{sample_name}_y'] += counts[1] #phage
            final_dict[arg_class][f'{sample_name}_z'] += counts[2] #unclassified
            final_dict[arg_class][f'{sample_name}_m'] += counts[3] #IS
            final_dict[arg_class][f'{sample_name}_n'] += counts[4] #IE

/tmp/ipykernel_266612/3887500067.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_chunk['ARG_class'] = filtered_chunk['ARG_class'].apply(preprocess_arg_class)
/tmp/ipykernel_266612/3887500067.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_chunk['ARG_class'] = filtered_chunk['ARG_class'].apply(preprocess_arg_class)
/tmp/ipykernel_266612/3887500067.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [13]:
final_df = pd.DataFrame(final_dict).T.fillna(0).astype(int)
final_df 

,S3WOOD_x,S3WOOD_y,S3WOOD_z,S3WOOD_m,S3WOOD_n,SS1-3_x,SS1-3_y,SS1-3_z,SS1-3_m,SS1-3_n,...,WWTP-0_x,WWTP-0_y,WWTP-0_z,WWTP-0_m,WWTP-0_n,WWTP-2_x,WWTP-2_y,WWTP-2_z,WWTP-2_m,WWTP-2_n
aminoglycoside,77,8,18,0,0,53,7,22,0,0,...,54,10,109,0,0,36,12,35,0,0
multidrug,66,1,15,0,0,37,2,11,0,0,...,71,27,196,0,0,42,1,21,0,0
tetracycline,14,4,8,0,0,19,3,12,0,0,...,23,16,53,0,0,7,3,16,0,0
MLS,0,1,4,0,0,7,1,4,0,0,...,28,14,59,0,0,4,2,3,0,0
diaminopyrimidine,8,1,1,0,0,3,0,1,0,0,...,6,3,8,0,0,2,3,4,0,0
disinfecting,6,1,1,0,0,8,1,0,0,0,...,10,1,16,0,0,2,0,7,0,0
macrolide,42,6,19,0,0,41,5,16,0,0,...,28,11,50,0,0,26,27,8,0,0
mupirocin,3,1,0,0,0,3,1,2,0,0,...,2,2,2,0,0,1,0,1,0,0
carbapenem,38,13,10,0,0,31,7,22,0,0,...,27,11,51,0,0,18,18,24,0,0
cephalosporin,20,1,6,0,0,13,2,15,0,0,...,8,5,19,0,0,10,5,4,0,0


In [15]:
final_df.to_csv('/lomi_home/gaoyang/software/CompRanking/tmp_DSR/DSR/CompRanking_result/ori_3_RGI_with_multidrug_adjust_cell_rpkg/MGE_carried_ARGs_type_count_final_result.tsv', sep='\t')